Dataset from: https://zindi.africa/competitions/digital-africa-plantation-counting-challenge

In [1]:
!pip install efficientnet_pytorch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=59dbedd46746d50bebde5ab9b716188138240d23eb7e266afef89580169b5cc9
  Stored in directory: /root/.cache/pip/wheels/96/3f/5f/13976445f67f3b4e77b054e65f7f4c39016e92e8358fe088db
Successfully built efficientnet_pytorch


In [2]:
Train = pd.read_csv('/kaggle/input/digital-africa-plantation-counting-challenge/Train.csv')
Test = pd.read_csv('/kaggle/input/digital-africa-plantation-counting-challenge/Test.csv')
SampleSubmission = pd.read_csv('/kaggle/input/digital-africa-plantation-counting-challenge/SampleSubmission.csv')

In [3]:
int(Train.shape[0]*0.85), int(Train.shape[0] - Train.shape[0]*0.85)

(1701, 300)

In [4]:
Train

,ImageId,Target
0,Id_jdqw9hlv6j.png,14.0
1,Id_6xtrolmuvc.png,18.0
2,Id_2m49sj3xd9.png,0.0
3,Id_9jwg5pcnn4.png,28.0
4,Id_vnm6e8n0p3.png,21.0
...,...,...
1997,Id_n2vxw7x9c5.png,14.0
1998,Id_1cx78gejxc.png,0.0
1999,Id_kbgnlekbjm.png,13.0
2000,Id_uzb88simbg.png,0.0


In [5]:
def otsu_threshold(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh
def refine_image(image):
    # apply Otsu's thresholding
    thresh = otsu_threshold(image)
    # define structuring element for erosion and dilation
    kernel = np.ones((3, 3), np.uint8)
    # perform erosion to remove small white regions or thin white lines
    eroded = cv2.erode(thresh, kernel, iterations=1)
    # perform dilation to remove small black regions or thin black lines
    dilated = cv2.dilate(eroded, kernel, iterations=1)

    return dilated

In [6]:
class d(Dataset):
    def __init__(self, csv_file, root_images, is_train=True, is_inference=False, transform=None ):
        
        self.root_images = root_images
        self.transform   = transform
        self.is_inference= is_inference
        
        if is_inference:
            self.csv_file = csv_file
        else:
            if is_train:
                self.csv_file = csv_file[:int(csv_file.shape[0]*0.85)].reset_index(drop=True)
            else:
                self.csv_file = csv_file[int(csv_file.shape[0]*0.85):].reset_index(drop=True)        
        
    def __len__(self):
        return self.csv_file.shape[0]
    
    def __getitem__(self, index):
        
        root_and_dir = self.csv_file['ImageId'][index]
        if not self.is_inference:
            label = self.csv_file['Target'][index]
    
        image = np.array(Image.open(os.path.join(self.root_images, root_and_dir)).convert('RGB'))
        
        if self.transform is not None:
            augmentations = self.transform(image=image)
            image         = augmentations['image']
            
        if not self.is_inference:
            return image, torch.as_tensor(label)
    
        return image

In [7]:
class PalmOilDataset(Dataset):
    def __init__(self, image_paths, labels=None, is_train=True, is_inference=False, transform=None):
        self.root = "/kaggle/input/digital-africa-plantation-counting-challenge/images/"
        self.image_paths = image_paths
        self.labels = labels
        self.is_inference = is_inference
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = np.array(Image.open(self.root+img_path).convert('RGB'))
        if not self.is_inference:
            label = self.labels[idx]
            
        if self.transform is not None:
            augmentations = self.transform(image=image)
            image         = augmentations['image']
            
        if not self.is_inference:
            return image, torch.as_tensor(label)
    
        return image

In [8]:
# LR = 1e-3
# BS = 4
# NE = 100
H  = 1024
W  = 1024
# train_file = Train
# test_file  = Test
# image_path = '/kaggle/input/digital-africa-plantation-counting-challenge/images'

In [9]:
normalize = A.Normalize(
    mean = [0.5,0.5,0.5],
    std  = [0.5,0.5,0.5], max_pixel_value=255
)

train_transform = A.Compose([
    A.Resize(H,W),
    A.Blur(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    normalize,
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(H,W),
    normalize,
    ToTensorV2(),
])

In [10]:
# train_ds     = d(train_file, image_path, is_train=True, is_inference=False, transform=train_transform )
# train_loader = DataLoader(train_ds, batch_size=BS, shuffle=True)

# val_ds     = d(train_file, image_path, is_train=False, is_inference=False, transform=val_transform )
# val_loader = DataLoader(val_ds, batch_size=BS, shuffle=False)

# test_ds     = d(test_file, image_path, is_train=False, is_inference=True, transform=val_transform )
# test_loader = DataLoader(test_ds, batch_size=BS, shuffle=False)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b3')
        self.fc1 = nn.Linear(1000, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128,1)
        
    def forward(self, image):
        x = self.model(image)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
#         x = self.fc(x)
        return x

In [12]:
class Model_Trainer:
    def __init__(self, Train, Test, Model, Dataset, train_transform, val_transform):
        self.Model = Model
        self.Dataset = Dataset
        self.Test = Test
        
        self.optimizer = optim.Adam(Model.parameters(), lr=1e-3)
        self.loss_fn = nn.MSELoss().to('cuda')
        self.es = 0
        self.X = Train['ImageId']
        self.y = Train['Target']
        self.train_transform = train_transform
        self.val_transform = val_transform
        self.NE = 10
        
    
    def trainer(self, train_loader, val_loader, encoder, NE, opt, loss_fn):
        best_loss = 999999999
        for epoch in range(NE):
            print('------------------------------- Epoch: '+str(epoch))
            encoder.train()
            for x,y in tqdm(train_loader):
                x = x.to('cuda').to(torch.float32)
                y = y.to(torch.float).unsqueeze(1).to('cuda')

                preds = encoder(x).to(torch.float)

                loss = loss_fn(preds, y)

                encoder.zero_grad()
                loss.backward()
                opt.step()

            running_loss = 0
            encoder.eval()
            with torch.no_grad():
                for x,y in tqdm(val_loader):
                    x = x.to('cuda').to(torch.float32)
                    y = y.to(torch.float).unsqueeze(1)

                    preds = encoder(x).to(torch.float)
                    running_loss += np.sqrt(mean_squared_error(preds.cpu(), y))
            print(f'Loss function: {running_loss/len(val_loader)}')
            new_loss = running_loss/len(val_loader)
            if new_loss < best_loss:
                best_loss = new_loss
                es   = 0
                filepath = 'encoder-%d.pkl' % epoch
                print("Saving CheckPoint")
                torch.save(encoder.state_dict(), filepath)

            else:
                es +=1

            if es == 3 :
                break
        return filepath, best_loss
    
    def inference(self, loader, filepath):
        inf = Net().to('cuda')
        inf.load_state_dict(torch.load(filepath))
        all_preds = np.array([])
        with torch.no_grad():
            for x in tqdm(loader):
                x = x.to('cuda').to(torch.float32)
                preds = inf(x).to(torch.float)
            
                all_preds = np.append(all_preds, preds.cpu())
            
        print('Done!')
        return all_preds                            
        
    def KFold_training(self):
        test_pred = []
        fold = KFold(n_splits=5, shuffle=True)
        i = 0
        for train_index, test_index in fold.split(self.X, self.y):
            X_train, X_test = self.X.iloc[train_index].reset_index(drop=True), self.X.iloc[test_index].reset_index(drop=True)
            y_train, y_test = self.y.iloc[train_index].reset_index(drop=True), self.y.iloc[test_index].reset_index(drop=True)
            train_ds = self.Dataset(X_train, y_train, is_train=True, is_inference=False, transform=self.train_transform)
            val_ds = self.Dataset(X_test, y_test, is_train=False, is_inference=False, transform=self.val_transform)
            train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
            val_loader = DataLoader(val_ds, batch_size=4, shuffle=False)
            
            filepath, best_loss = self.trainer(train_loader, val_loader,self.Model,self.NE,self.optimizer,self.loss_fn)
            print(f"Fold {i}, Best Loss:{best_loss}")
            i += 1
            test_ds     = self.Dataset(self.Test['ImageId'], is_train=False, is_inference=True, transform=self.val_transform )
            test_loader = DataLoader(test_ds, batch_size=4, shuffle=False)
            preds = self.inference(test_loader, filepath)
            test_pred.append(preds)
        return test_pred

In [13]:
encoder = Net().to('cuda')


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


In [14]:
model_trainer = Model_Trainer(Train, Test, encoder, PalmOilDataset, train_transform, val_transform)

In [15]:
predictions = model_trainer.KFold_training()

------------------------------- Epoch: 0


100%|██████████| 101/101 [00:44<00:00,  2.26it/s]


Loss function: 4.076897943078881
Saving CheckPoint
------------------------------- Epoch: 1


100%|██████████| 101/101 [00:37<00:00,  2.71it/s]


Loss function: 2.4727023202593963
Saving CheckPoint
------------------------------- Epoch: 2


...

Loss function: 2.174665236771107
------------------------------- Epoch: 3


100%|██████████| 100/100 [00:34<00:00,  2.89it/s]


Loss function: 2.5207030891999604
Fold 4, Best Loss:1.452349308207631
Loaded pretrained weights for efficientnet-b3


100%|██████████| 215/215 [01:21<00:00,  2.65it/s]

Done!


In [40]:
preds = (predictions[0]+predictions[1]+predictions[2]+predictions[3]+predictions[4])/5

In [41]:
Test['Target'] = preds

In [42]:
Test['Target'] = Test['Target'].apply(lambda x: int(x))
Test

,ImageId,Target
0,Id_ohk78h9ld8.png,0
1,Id_eeyj2u4j7y.png,0
2,Id_wsd7vx2ifa.png,14
3,Id_6vfneamaoh.png,11
4,Id_9wil3575fv.png,17
...,...,...
853,Id_lmvuv1pm3a.png,0
854,Id_ez9lb2o6b1.png,37
855,Id_jeou44iven.png,0
856,Id_341bsipcnk.png,9


In [43]:
Test.to_csv('subensemble_intall.csv', index=False)

In [31]:
Test

,ImageId,Target
0,Id_ohk78h9ld8.png,0
1,Id_eeyj2u4j7y.png,0
2,Id_wsd7vx2ifa.png,14
3,Id_6vfneamaoh.png,12
4,Id_9wil3575fv.png,16
...,...,...
853,Id_lmvuv1pm3a.png,0
854,Id_ez9lb2o6b1.png,37
855,Id_jeou44iven.png,0
856,Id_341bsipcnk.png,10
